___
# Ciência dos Dados - PROJETO 2

___
## Leonardo Bhering Gonçalves

## Felipe da Silva Pereira

___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [145]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from numpy import arange
from sklearn.model_selection import train_test_split

In [146]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\eletr\Google Drive\Insper\Ciência dos dados\Kelly\cddp2


In [147]:
df = pd.read_excel('spamhamV2.xlsx')

df['Email']=df.Email.str.replace(':','')
df['Email']=df.Email.str.replace('"','')
df['Email']=df.Email.str.replace("'","")
df['Email']=df.Email.str.replace('(','')
df['Email']=df.Email.str.replace(')','')
df['Email']=df.Email.str.replace('\n','')
df['Email']=df.Email.str.replace(',','')
df['Email']=df.Email.str.replace('.','')
df['Email']=df.Email.str.lower()

In [148]:
df.dtypes

Email    object
Class    object
dtype: object

In [149]:
treinamento, teste = train_test_split(df)

In [150]:
#De outro modo

df['Email']=df.Email.str.replace(':','')
df['Email']=df.Email.str.replace('"','')
df['Email']=df.Email.str.replace("'","")
df['Email']=df.Email.str.replace('(','')
df['Email']=df.Email.str.replace(')','')
df['Email']=df.Email.str.replace('\n','')
df['Email']=df.Email.str.replace(',','')
df['Email']=df.Email.str.replace('.','')
df['Email']=df.Email.str.lower()

In [151]:
#Dividir cada email em palavras
df['Email']=df['Email'].str.split(pat=None,n=-1,expand=False)

In [152]:
#Agora podemos separar em dois arquivos distintos, teste e treinamento
basetreinamento, baseteste=df.sample(n=None, frac=0.75, replace=False, weights=None, random_state=None, axis=None), df.sample(n=None, frac=0.25, replace=False, weights=None, random_state=None, axis=None)


#basetreinamento.to_csv('treinamento.csv')
#baseteste.to_csv('teste.csv')

In [153]:
treinamento=treinamento.replace({'@':''}, regex=True)
treinamento=treinamento.replace({'()':''}, regex=True)
treinamento=treinamento.replace({':':''}, regex=True)
treinamento=treinamento.replace({'#':''}, regex=True)
treinamento=treinamento.replace({';':''}, regex=True)
treinamento=treinamento.replace({'':''}, regex=True)
treinamento=treinamento.replace({',':''}, regex=True)
treinamento=treinamento.replace({'"':''}, regex=True)
treinamento=treinamento.replace({'  ':''}, regex=True)

___
## 3. Classificador Naive-Bayes

In [154]:
spam=treinamento[treinamento.Class=='spam']
prob_spam=len(spam)/len(treinamento['Class'])
print('Probabilidade de uma mensagem ser SPAM:', prob_spam)

Probabilidade de uma mensagem ser SPAM: 0.13711414213926776


In [155]:
ham=treinamento[treinamento.Class=='ham']
prob_ham=len(ham)/len(treinamento['Class'])
print('Probabilidade de uma mensagem ser HAM:', prob_ham)

Probabilidade de uma mensagem ser HAM: 0.8628858578607322


In [156]:
#Frequência por palavra dentro do df treinamento
total=treinamento['Email'].sum(axis=None, skipna=None, level=None, numeric_only=None)
n_emails=len(treinamento) #Numero de emails
serie_email = pd.Series(total)
email_vc = serie_email.value_counts()
#Probabilidade total de cada palavra no dataframe
prob_palavra=(email_vc+1)/(len(serie_email) + len(email_vc))



In [157]:
serie_class = pd.Series(treinamento['Class'])
class_vc = serie_class.value_counts()
#Probabilidade total de relevancia no dataframe
prob_relevancia=(class_vc+1)/(len(serie_class) + len(class_vc))

In [158]:
#FILTRANDO HAM
treino_ham=treinamento[treinamento.Class=='ham']

treino_ham=treino_ham['Email'].sum(axis=None, skipna=None, level=None, numeric_only=None)
serie_ham = pd.Series(treino_ham)
ham_vc = serie_ham.value_counts()
#Probabilidade de cada palavra pertencer a um email HAM
string_ham=(ham_vc+1)/(len(serie_ham) + len(serie_email))

In [159]:
#FILTRANDO SPAM
treino_spam=treinamento[treinamento.Class=='spam']

treino_spam=treino_spam['Email'].sum(axis=None, skipna=None, level=None, numeric_only=None)
serie_spam = pd.Series(treino_spam)
spam_vc = serie_spam.value_counts()
#Probabilidade de cada palavra pertencer a um email SPAM
string_spam=(spam_vc+1)/(len(serie_spam) + len(serie_email))

In [170]:
#Função de Naive-Bayes:

def Bayes(Email):
    vspam=prob_spam
    vham=prob_ham
    for x in Email:
        if x in string_spam:
            vspam*=string_spam[x]
        else:
            vspam*=1/(len(email_vc))
    for x in Email:
        if x in string_ham:
            vham*=string_ham[x]
        else:
            vham*=1/(len(email_vc))
            
    if vspam > vham:
        return("spam")
    elif vham>vspam:
        return("ham")
            
        

In [171]:
#Validando a classificação obtida por Naive-Bayes - Execução do teste

def valida(teste):
    texto=teste.Email
    return Bayes(texto)

Teste1 = baseteste.apply(valida,axis=1)
baseteste["Classificação"] = Teste1

In [172]:
baseteste['Class'].value_counts()

ham     1209
spam     184
Name: Class, dtype: int64

In [173]:
baseteste['Classificação'].value_counts()

ham    1393
Name: Classificação, dtype: int64

In [177]:
baseteste.loc[(baseteste.Class == "ham") & (baseteste.Classificação == "ham"),"Corrigido"] = "negativo verdadeiro"
baseteste.loc[(baseteste.Class == "spam") & (baseteste.Classificação == "spam"),"Corrigido"] = "positivo verdadeiro"
baseteste.loc[(baseteste.Class == "spam") & (baseteste.Classificação == "ham"),"Corrigido"] = "positivo falso"
baseteste.loc[(baseteste.Class == "ham") & (baseteste.Classificação == "spam"),"Corrigido"] = "negativo falso"

Positivo_Verdadeiro = baseteste.Corrigido[baseteste.Corrigido == "negativo verdadeiro"].value_counts()
Negativo_Verdadeiro = baseteste.Corrigido[baseteste.Corrigido == "positivo verdadeiro"].value_counts()
Positivo_Falso = baseteste.Corrigido[baseteste.Corrigido == "positivo falso"].value_counts()
Negativo_Falso = baseteste.Corrigido[baseteste.Corrigido == "negativo falso"].value_counts()

Probabilidade_Positivo_Verdadeiro = (Positivo_Verdadeiro/len(baseteste)*100)
print("Probabilidade de um positivo verdadeiro:  " +str(Probabilidade_Positivo_Verdadeiro))
Probabilidade_Negativo_Verdadeiro = (Negativo_Verdadeiro/len(baseteste)*100)
print("Probabilidade de um negativo verdadeiro:  " +str(Probabilidade_Negativo_Verdadeiro))
Porbabilidade_Positivo_Falso = (Positivo_Falso/len(baseteste)*100)
print("Probabilidade de um positivo falso:  " +str(Porbabilidade_Positivo_Falso))
Probabilidade_Negativo_Falso = (Negativo_Falso/len(baseteste)*100)
print("Probabilidade de um negativo falso:  " +str(Probabilidade_Negativo_Falso))

Probabilidade de um positivo verdadeiro:  negativo verdadeiro    86.791098
Name: Corrigido, dtype: float64
Probabilidade de um negativo verdadeiro:  Series([], Name: Corrigido, dtype: float64)
Probabilidade de um positivo falso:  positivo falso    13.208902
Name: Corrigido, dtype: float64
Probabilidade de um negativo falso:  Series([], Name: Corrigido, dtype: float64)


___
## 4. Curva ROC

___
## 5. Qualidade do Classificador alterando a base de treinamento